In [68]:
import os
from dotenv import load_dotenv
import pandas as pd
from datetime import datetime
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import json
from supabase import create_client
import time

load_dotenv()



True

In [69]:
# Connection to Supabase
url_supabase = os.getenv("SUPABASE_URL")
key = os.getenv("SUPABASE_KEY")
supabase = create_client(url_supabase, key)

In [70]:
db_data = supabase.table("idx_key_stats").select("").execute()
df_db_data = pd.DataFrame(db_data.data)

df_db_data.head()

,symbol,recommendation_mean,point_summaries,updated_on,employee_num,holders_breakdown,intrinsic_value,forward_eps
0,ACRO.JK,NaN,None,2024-06-10T14:32:22+00:00,25.0,None,NaN,NaN
1,RMBA.JK,NaN,"[{'name': 'value', 'point': 5, 'maxpoint': 20}...",2024-05-24T06:24:51+00:00,978.0,"{'% of Float Held by Institutions': '0.00%', '...",234.772165,NaN
2,AALI.JK,4.3,"[{'name': 'value', 'point': 16, 'maxpoint': 21...",2024-06-10T14:32:22+00:00,30025.0,None,8158.263096,635.05
3,DGNS.JK,NaN,"[{'name': 'value', 'point': 11.5, 'maxpoint': ...",2024-06-10T14:32:22+00:00,64.0,None,8.908110,NaN
4,EPAC.JK,NaN,"[{'name': 'value', 'point': 13.5, 'maxpoint': ...",2024-06-10T14:32:22+00:00,160.0,None,NaN,NaN


In [71]:
# Add new column for rating data
df_db_data['technical_rating'] = np.nan
df_db_data['analyst_rating']  = np.nan

cols = df_db_data.columns.tolist()
df_db_data.head()

,symbol,recommendation_mean,point_summaries,updated_on,employee_num,holders_breakdown,intrinsic_value,forward_eps,technical_rating,analyst_rating
0,ACRO.JK,NaN,None,2024-06-10T14:32:22+00:00,25.0,None,NaN,NaN,NaN,NaN
1,RMBA.JK,NaN,"[{'name': 'value', 'point': 5, 'maxpoint': 20}...",2024-05-24T06:24:51+00:00,978.0,"{'% of Float Held by Institutions': '0.00%', '...",234.772165,NaN,NaN,NaN
2,AALI.JK,4.3,"[{'name': 'value', 'point': 16, 'maxpoint': 21...",2024-06-10T14:32:22+00:00,30025.0,None,8158.263096,635.05,NaN,NaN
3,DGNS.JK,NaN,"[{'name': 'value', 'point': 11.5, 'maxpoint': ...",2024-06-10T14:32:22+00:00,64.0,None,8.908110,NaN,NaN,NaN
4,EPAC.JK,NaN,"[{'name': 'value', 'point': 13.5, 'maxpoint': ...",2024-06-10T14:32:22+00:00,160.0,None,NaN,NaN,NaN,NaN


In [72]:
# Get symbol data
symbol_list = df_db_data['symbol'].tolist()
symbol_list

# Remove the .JK
for i in range (len(symbol_list)):
  symbol_list[i] = symbol_list[i].replace(".JK", "")

symbol_list[:5]

['ACRO', 'RMBA', 'AALI', 'DGNS', 'EPAC']

In [104]:
# Scraping data
BASE_URL = "https://www.tradingview.com/chart/?symbol=IDX%3A"

def get_url_page(symbol:str) -> str:
    return f"{BASE_URL}{symbol}"

def scrap_page(url: str) :
    driver = webdriver.Chrome()
    driver.get(url)
    try:
        loader = WebDriverWait(driver, 1).until(
            EC.invisibility_of_element((By.ID, "loader"))
        )
        print(f"Successfully get element from URL: {url}")
        return driver
    except:
      print(f"Fail scraping from URL: {url}")
      print("Loader did not disappear in time")
      driver.quit()
      return None
    
def scrap_rating_data(symbol: str) -> dict:
    url = get_url_page(symbol)
    loader = scrap_page(url)
    result_data = dict()
    result_data['symbol'] = symbol
    result_data['updated_on'] = (datetime.now()).strftime("%Y-%m-%d %H:%M:%S")
    print(result_data)

    # Ignore if element is None
    if (loader is not None):
      items = loader.find_elements(By.CLASS_NAME, "button-vll9ujXF")
      for item in items:
        
        # Getting technical
        if (item.text == "More technicals"):
          item.click()

        # Getting Analyst Rating
        if (item.text == "See forecast"):
           item.click()
       


In [105]:
scrap_rating_data("AMMN")

Successfully get element from URL: https://www.tradingview.com/chart/?symbol=IDX%3AAMMN
{'symbol': 'AMMN', 'updated_on': '2024-06-26 16:00:36'}
